In [9]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

import json

api_key = 'RGAPI-2bf73814-83af-4540-a2e0-46eee1c8271f'
watcher = LolWatcher(api_key=api_key)
my_region = 'na1'
me = watcher.summoner.by_name(my_region, 'yENatividad')

In [56]:
def json_data_mask(dic):
    keys_to_remove = []
    for k,v in dic.items():
        if type(v) is dict:
            keys_to_remove = keys_to_remove + [k]
    return keys_to_remove


def clean_timeframe(timeline, frame=5):
    players_list = list(timeline['info']['frames'][frame]['participantFrames'].values())
    keys_to_remove = json_data_mask(players_list[0])
    keys_to_remove += ['currentGold', 'goldPerSecond', 'participantId', 'totalDamageDone', 'totalDamageDoneToChampions', 'totalDamageTaken']
    for player in players_list:
        if 'damageStats' in player.keys():
            damage_stat = player['damageStats']
            player.update(damage_stat)
        for key in keys_to_remove:
            if key in player.keys():
                player.pop(key)
    return players_list

def add_creep_score(timeframe):
    if type(timeframe) is not list:
        raise TypeError("Input timeframe must be a list")
    elif type(timeframe[0]) is not dict:
        raise TypeError("Elements of inputted timeframe are not dicts.")
    
    for player in timeframe:
        if 'jungleMinionsKilled' in player.keys() and 'minionsKilled' in player.keys():
            player['creepScore'] = player['jungleMinionsKilled'] +player['minionsKilled']
            player.pop('jungleMinionsKilled')
            player.pop('minionsKilled')
        elif 'jungleMinionsKilled' not in player.keys() and 'minionsKilled' not in player.keys():
            if 'creepScore' not in player.keys():
                raise ValueError("Missing information to construct creep score for a player.")
        else:
            raise ValueError("Missing information to construct creep score for a player.")
    
    return timeframe

def process_timeframe(timeline, win, frame=10, matchid=None):
    cleaned = clean_timeframe(timeline, frame)
    cleaned = add_creep_score(cleaned)
    final_dict = {}
    for i in range(5):
        for key in list(cleaned[0].keys()):
            final_dict[key+'_'+str(i)] = cleaned[i][key] - cleaned[i+5][key]
    final_dict['matchId'] = matchid if matchid else 'UNKNOWN'
    final_dict['win'] = bool(win)
    return final_dict
    

## Match Summary

In [32]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'])
latest_match = watcher.match.by_id(my_region, my_matches[0])

In [36]:
players_list = latest_match['info']['participants']
rows = []

# Create mask for non-numerical dict fields 
keys_to_remove = json_data_mask(players_list[0])

# Construct dictionary list that prepares for pandas DataFrame
for player in players_list:
    for key in keys_to_remove:
        if key in player.keys():
            player.pop(key)
    rows += [player]

# Pandas Dataframe
match_summary = pd.DataFrame(rows)
match_summary 

,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,...,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,9,1,0,20440,18,50,Swain,0,2,0,...,4000,0,0,11,0,10,0,1,9,False
1,14,0,0,18112,17,33,Rammus,0,13,0,...,2614,0,0,11,0,35,8,4,13,False
2,8,0,0,20408,18,105,Fizz,0,2,3070,...,1854,1,1,11,0,22,0,1,14,False
3,6,0,0,23534,18,119,Draven,0,5,0,...,603,0,0,11,0,33,3,2,13,False
4,14,0,0,18179,17,89,Leona,0,13,0,...,1244,0,0,11,0,69,9,9,35,False
5,8,0,1,20936,18,82,Mordekaiser,0,6,5423,...,848,2,3,1,0,38,4,11,5,True
6,14,1,1,27356,18,102,Shyvana,0,6,1166,...,1991,2,2,1,0,43,2,11,3,True
7,13,0,2,19674,18,99,Lux,0,2,3853,...,392,1,5,1,0,27,0,6,10,True
8,16,0,0,18346,17,81,Ezreal,0,3,9760,...,2241,6,7,1,0,15,0,3,9,True
9,26,0,0,16289,16,267,Nami,0,13,2201,...,613,0,6,1,0,100,11,9,46,True


## Match Timeline 10min

In [57]:
latest_timeline = watcher.match.timeline_by_match(my_region, my_matches[0])

In [58]:
pd.DataFrame([process_timeframe(latest_timeline, False, 10)])

,level_0,timeEnemySpentControlled_0,totalGold_0,xp_0,magicDamageDone_0,magicDamageDoneToChampions_0,magicDamageTaken_0,physicalDamageDone_0,physicalDamageDoneToChampions_0,physicalDamageTaken_0,...,magicDamageTaken_4,physicalDamageDone_4,physicalDamageDoneToChampions_4,physicalDamageTaken_4,trueDamageDone_4,trueDamageDoneToChampions_4,trueDamageTaken_4,creepScore_4,matchId,win
0,0,88650,-910,-366,-1886,-759,759,-2869,79,-254,...,441,899,67,-50,1418,356,-90,5,,False
